In [7]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import ibm_boto3
from ibm_botocore.client import Config, ClientError
import time
import numpy as np
from io import BytesIO
import urllib.request as req
import wget
from PIL import Image
from datetime import datetime
import io as libio
import sched, time
from datetime import datetime, timedelta
from threading import Timer
import xml.dom.minidom
from xml.etree import ElementTree
import os

file_path = os.getcwd()
print(file_path)

s = sched.scheduler(time.time, time.sleep)

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.eu-gb.cloud-object-storage.appdomain.cloud"
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_API_KEY_ID = "BD98N-vqonh983XVsKzNhIRx62oKMGZDodhNTL4ou9Tp" 
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/5654a1f2157d42d482817601f6b4f60c:1b9a82e2-2215-4c79-ac53-7f021a893306:bucket:cos.deeplearning.walekova" 
COS_BUCKET = 'accu-clouds'

cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

def upload_large_file(item_name):
    print("Starting large file upload for {0} to bucket: {1}".format(item_name, COS_BUCKET))

    # set the chunk size to 5 MB
    part_size = 1024 * 1024 * 5

    # set threadhold to 5 MB
    file_threshold = 1024 * 1024 * 5

    # set the transfer threshold and chunk size in config settings
    transfer_config = ibm_boto3.s3.transfer.TransferConfig(
        multipart_threshold=file_threshold,
        multipart_chunksize=part_size
    )

    # create transfer manager
    transfer_mgr = ibm_boto3.s3.transfer.TransferManager(cos, config=transfer_config)

    try:
        # initiate file upload
        future = transfer_mgr.upload(file_path, COS_BUCKET, item_name)

        # wait for upload to complete
        future.result()

        #print ("Large file upload complete!")
    except Exception as e:
        print("Unable to complete large file upload: {0}".format(e))
    finally:
        transfer_mgr.shutdown()   
        
def writeImage(tod):
    
    request = "https://radar.weather.gov/ridge/Conus/RadarImg/latest.gif"
      
    response = requests.get(request)
    img = Image.open(BytesIO(response.content))

    # Create buffer to hold jpeg representation of image in 'io.BytesIO' object
    bufImage = libio.BytesIO()
    # Store jpeg representation of image in buffer
    img.save(bufImage,"PNG") 
    # Rewind the buffer to beginning
    bufImage.seek(0)

    file = "USA_" + str(tod) + ".gif"

    # Provide the object to the Body parameter of put_object to write image to COS
    cos.put_object(Bucket= COS_BUCKET, Body = bufImage , Key = file, ContentType = 'image/gif')
    
    cities_df = pd.read_csv('us_cities_allinfo.csv')
    
    data_col = ['key','city', 'state', 'lat', 'lon', 'time', 'timeZone', 'obsDaylight', 'currentGmtOffset', 'timeZoneAbbreviation', \
       'observationtime', 'pressurestate', 'temperature', 'realfeel', 'humidity', 'weathertext', 'weathericon', 'windgusts', \
       'windspeed', 'winddirection', 'visibility', 'precip', 'uvindex', 'dewpoint', 'cloudcover']
    
    weather_24_us = pd.DataFrame(columns=data_col)
    
    for index, row in cities_df.iterrows():
        get_locationKey = row['Key']
    
        current = requests.get("http://samsungmobile.accu-weather.com/widget/samsungmobile/weather-data.asp?metric=1&location=cityId%3A"+ str(get_locationKey))
        root = ElementTree.fromstring(current.content)
    
        conditions = [[get_locationKey ,root[1][0].text, root[1][1].text, root[1][2].text, root[1][3].text, root[1][4].text, root[1][5].text, \
                  root[1][6].text, root[1][7].text, root[1][8].text, root[3][1].text, root[3][2].text, root[3][3].text, \
                  root[3][4].text, root[3][5].text, root[3][6].text, root[3][7].text, root[3][8].text, root[3][9].text, \
                  root[3][10].text, root[3][11].text, root[3][12].text, root[3][13].text, root[3][14].text, root[3][15].text]]
    
        current_df = pd.DataFrame(conditions, columns=data_col)
        weather_24_us = pd.concat([weather_24_us, current_df])
    
    file = "USA_" + str(tod) + ".csv"
    weather_24_us.to_csv(file)
    #upload_large_file(file)
    weather_24_us.drop(weather_24_us.index, inplace=True)
    
def do_something(sc): 
    # do your stuff
    dt_object = datetime.now()
    tod = dt_object.strftime('%y%m%d%H%M')
    writeImage(tod)

    s.enter(720, 1, do_something, (sc,))

s.enter(720, 1, do_something, (s,))
s.run()



C:\Users\walek\Desktop\w251\Project


C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

...

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati

C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning
C:\Users\walek\Anaconda3\lib\site-packages\ibm_botocore\vendored\requests\api.py:67: Deprecati